In [3]:
from pinkrigs_tools.dataset.query import load_data, queryCSV
from pinkrigs_tools.utils.spk_utils import bombcell_sort_units
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from datetime import datetime
import time
import seaborn as sns
from scipy import stats, signal
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from neuropixel_utils import *
from correlation_utils import *
%load_ext autoreload
%autoreload 2

In [ ]:
from find_mouse_ids import batch_analyze_delta_spectrograms

summary_batch_results = batch_analyze_delta_spectrograms(
    subject_ids=['AV005', 'AV006', 'AV007', 'AV008', 'AV009', 'AV013', 'AV014', 'AV015', 'AV020', 'AV021', 'AV023', 'AV028', 'AV030', 'AV035'],
    output_base_dir=r"C:\Users\Experiment\Projects\video_conversions\subjects\batch_analysis"
)
# Can be run with single subjid or list of subjid's, using ['AV043', 'AV044'] for example.

### Helper Functions:


In [25]:
# Query all subjects with valid sessions
exp = queryCSV(
    subject='all',
    expDate='all', 
    expDef=['postactive', 'spontaneous'],
    checkSpikes='1'
)

if not exp.empty:
    # Get unique subject IDs
    valid_subjects = exp['subject'].unique()
    
    print(f"Found {len(valid_subjects)} subjects with valid sessions:")
    print(f"Subject IDs: {list(valid_subjects)}")
    
    # Optional: Show session counts per subject
    session_counts = exp['subject'].value_counts()
    print(f"\nSession counts per subject:")
    for subject, count in session_counts.items():
        print(f"  {subject}: {count} sessions")
    
    # Optional: Show breakdown by experiment type
    print(f"\nBreakdown by experiment type:")
    exp_type_counts = exp.groupby(['subject', 'expDef']).size().unstack(fill_value=0)
    print(exp_type_counts)
    
else:
    print("No valid sessions found!")
    valid_subjects = []

# Use this list for batch processing
print(f"\nReady for batch processing with: {list(valid_subjects)}")

Found 39 subjects with valid sessions:
Subject IDs: ['AV005', 'AV006', 'AV007', 'AV008', 'AV009', 'AV013', 'AV014', 'AV015', 'AV020', 'AV021', 'AV023', 'AV024', 'AV025', 'AV028', 'AV030', 'AV034', 'AV035', 'CB016', 'CB018', 'CB020', 'EB014', 'FT019', 'FT022', 'FT025', 'FT027', 'FT030', 'FT031', 'FT032', 'FT033', 'FT035', 'FT038', 'FT039', 'AV043', 'AV049', 'GB002', 'GB011', 'GB012', 'EB036', 'EB037']

Session counts per subject:
  AV009: 363 sessions
  AV008: 151 sessions
  AV007: 143 sessions
  AV013: 135 sessions
  AV015: 134 sessions
  GB012: 102 sessions
  AV014: 78 sessions
  AV049: 77 sessions
  AV005: 62 sessions
  AV023: 62 sessions
  AV006: 55 sessions
  AV043: 53 sessions
  EB014: 36 sessions
  AV020: 33 sessions
  AV021: 33 sessions
  AV035: 32 sessions
  FT025: 25 sessions
  AV028: 20 sessions
  GB002: 19 sessions
  AV025: 17 sessions
  FT027: 16 sessions
  FT019: 13 sessions
  AV030: 13 sessions
  GB011: 11 sessions
  FT030: 11 sessions
  AV034: 10 sessions
  FT039: 10 ses

In [ ]:
# See what experiment types are available for a subjid
from pinkrigs_tools.dataset.query import queryCSV

# Get all experiments for AV043 to see what expDef values are available
exp_all = queryCSV(
    subject='AV043',
    expDate='all',  # Get all dates
    expDef='all'    # Get all experiment definitions
)

# Look at unique experiment definitions
unique_expdefs = exp_all['expDef'].unique()
print(f"Available experiment definitions for:")
for expdef in unique_expdefs:
    print(f"- {expdef}")

Available experiment definitions for:
- spontaneousActivity


In [1]:
# Find valid sessions (filter for exp Parameter and valid ephys recording)

from pinkrigs_tools.dataset.query import queryCSV, load_data
import os
import pandas as pd

# Query all experiments for AV043 with spontaneousActivity
exp_all = queryCSV(
    subject='AV043',
    expDate='all',  # Get all dates
    expDef='AVPassive_ckeckerboard_postactive'    # Get experiments with this expDef
)

print(f"Found {len(exp_all)} total AVPassive_ckeckerboard_postactive sessions for AV043")

# Filter for valid sessions (those with existing experiment and ephys folders)
valid_sessions = []

for _, row in exp_all.iterrows():
    # Check if required folder paths exist
    exp_folder_exists = pd.notna(row.get('expFolder')) and os.path.exists(row.get('expFolder', ''))
    ephys_probe0_exists = pd.notna(row.get('ephysPathProbe0')) and os.path.exists(row.get('ephysPathProbe0', ''))
    ephys_probe1_exists = pd.notna(row.get('ephysPathProbe1')) and os.path.exists(row.get('ephysPathProbe1', ''))
    
    # Consider session valid if experiment folder exists and at least one probe has ephys data
    if exp_folder_exists and (ephys_probe0_exists or ephys_probe1_exists):
        session_info = {
            'subject': row['subject'],
            'expDate': row['expDate'],
            'expNum': row.get('expNum', 'N/A'),
            'expFolder': row.get('expFolder'),
            'ephysPathProbe0': row.get('ephysPathProbe0') if ephys_probe0_exists else None,
            'ephysPathProbe1': row.get('ephysPathProbe1') if ephys_probe1_exists else None,
            'has_probe0': ephys_probe0_exists,
            'has_probe1': ephys_probe1_exists
        }
        valid_sessions.append(session_info)
        
        print(f"✓ Valid: {row['expDate']} (expNum: {row.get('expNum', 'N/A')}) - "
              f"Probe0: {'Yes' if ephys_probe0_exists else 'No'}, "
              f"Probe1: {'Yes' if ephys_probe1_exists else 'No'}")
    else:
        print(f"✗ Invalid: {row['expDate']} (expNum: {row.get('expNum', 'N/A')}) - "
              f"ExpFolder: {'Yes' if exp_folder_exists else 'No'}, "
              f"Probe0: {'Yes' if ephys_probe0_exists else 'No'}, "
              f"Probe1: {'Yes' if ephys_probe1_exists else 'No'}")

print(f"\nFound {len(valid_sessions)} valid sessions out of {len(exp_all)} total sessions")

# Convert to DataFrame for easier handling
valid_sessions_df = pd.DataFrame(valid_sessions)
print("\nValid sessions summary:")
print(valid_sessions_df[['expDate', 'expNum', 'has_probe0', 'has_probe1']])

# Now you can easily select any valid session for analysis
if len(valid_sessions) > 0:
    # Example: Select the first valid session
    selected_session = valid_sessions[0]
    
    SUBJECT = selected_session['subject']
    EXP_DATE = selected_session['expDate']
    EXP_NUM = selected_session['expNum']
    
    exp_kwargs = {
        'subject': [SUBJECT],
        'expDate': EXP_DATE,
        'expNum': EXP_NUM,
    }
    
    print(f"\nSelected session for analysis: {SUBJECT} on {EXP_DATE} experiment {EXP_NUM}")
    print(f"Experiment folder: {selected_session['expFolder']}")
    if selected_session['has_probe0']:
        print(f"Probe0 ephys folder: {selected_session['ephysPathProbe0']}")
    if selected_session['has_probe1']:
        print(f"Probe1 ephys folder: {selected_session['ephysPathProbe1']}")
    
    # Test loading data for the selected session
    try:
        data_name_dict = {'events':{'_av_trials':'table'}}
        recordings = load_data(data_name_dict=data_name_dict, **exp_kwargs)
        exp_info = queryCSV(**exp_kwargs)
        print(f"✓ Successfully loaded data for {SUBJECT} on {EXP_DATE} experiment {EXP_NUM}")
    except Exception as e:
        print(f"✗ Error loading data: {e}")
else:
    print("No valid sessions found!")

Found 39 total AVPassive_ckeckerboard_postactive sessions for AV043
✓ Valid: 2024-01-26 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-01-29 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-01-30 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-01-31 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-02-01 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-02-02 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-02-05 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-02-06 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-02-07 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-02-08 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-02-09 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-02-12 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-02-13 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-02-14 (expNum: 3) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-02-15 (expNum: 2) - Probe0: Yes, Probe1: Yes
✓ Valid: 2024-02-16 (expNum: 2) - Probe0: Yes, 